# Get Raw GA360 Data

In [1]:
%load_ext autoreload
%autoreload 2

::: {.content-hidden}
Import necessary Python modules
:::

In [2]:
import os
import sys
from datetime import datetime, timedelta

import pandas as pd
import pytz

::: {.content-hidden}
Get relative path to project root directory
:::

In [3]:
PROJ_ROOT_DIR = os.path.join(os.pardir)
src_dir = os.path.join(PROJ_ROOT_DIR, "src")
sys.path.append(src_dir)

::: {.content-hidden}
Import custom Python modules
:::

In [4]:
%aimport bigquery_auth_helpers
from bigquery_auth_helpers import auth_to_bigquery

%aimport sql_helpers
import sql_helpers as sqlh

%aimport transform_helpers
import transform_helpers as th

%aimport utils
import utils as ut

## About

Retrieve GA360 tracking data from public [sample data of Google merchandise store transactions](https://support.google.com/analytics/answer/7586738?hl=en&ref_topic=3416089&sjid=2689173516056332244-NA#zippy=%2Cin-this-article) and upload to BigQuery dataset table in personal Google Clout account.

## User Inputs

Define the following

1. start date and the number of days following this date for which raw data should be queried
2. list of nested columns in raw GA360 tracking data that is accessible in the sample merchandise store dataset

In [5]:
#| echo: true
# 1. start date and number of following days
date_start = datetime(2016, 11, 15)
num_days = 7

# 2. list of nested columns in raw data
nested_cols = [
    'totals',
    'trafficSource',
    'device',
    'geoNetwork',
    'customDimensions',
    'hits',
]

Get end date for which raw data should be queried

In [6]:
date_start_str = date_start.strftime("%Y%m%d")
date_end_str = (date_start + timedelta(days=num_days-1)).strftime("%Y%m%d")

::: {.content-hidden}
Get path to data sub-folders
:::

In [7]:
data_dir = os.path.join(PROJ_ROOT_DIR, "data")
raw_data_dir = os.path.join(data_dir, "raw")

Set datatypes for non-nested fields in raw GA360 tracking data

In [8]:
dtypes_dict_raw = {
    "visitorId": pd.Int64Dtype(),
    "visitNumber": pd.Int64Dtype(),
    "visitId": pd.Int64Dtype(),
    "visitStartTime": pd.Int64Dtype(),
    "date": pd.StringDtype(),
    "fullVisitorId": pd.StringDtype(),
    "userId": pd.StringDtype(),
    "channelGrouping": pd.StringDtype(),
    "socialEngagementType": pd.StringDtype(),
}

Define helper function export a `DataFrame` to a local `parquet` file with `gzip` compression

In [9]:
def load(df: pd.DataFrame, fpath: str) -> None:
    """Save DataFrame to parquet file."""
    df.to_parquet(fpath, index=False, compression='gzip', engine='pyarrow')

::: {.content-hidden}
## Authenticate to `BigQuery`
:::

In [10]:
gcp_auth_dict = auth_to_bigquery(raw_data_dir)

## Get Data

Check number of observations in the single batch of raw GA360 tracking data that covers the start and end dates defined above

In [11]:
query_infer = f"""
              SELECT COUNT(fullVisitorId) AS num_visitors,
                     COUNT(visitId) AS num_visits,
                     COUNT(visitNumber) AS num_visitNumbers,
                     COUNT(visitStartTime) AS num_visitStartTime
              FROM `data-to-insights.ecommerce.web_analytics`
              WHERE date BETWEEN '{date_start_str}' AND '{date_end_str}'
              """
df = th.extract_data(query_infer, gcp_auth_dict)
df

Query execution start time = 2023-05-30 16:10:50.449...done at 2023-05-30 16:10:51.968 (1.519 seconds).
Query returned 1 rows


,num_visitors,num_visits,num_visitNumbers,num_visitStartTime
0,29396,29396,29396,29396


Read single batch of raw GA360 tracking data using a simple ETL workflow

1. extract raw data covering the start and end dates defined above
2. transform raw data by changing datatypes
3. export transformed data to local disk

In [12]:
query_infer = f"""
              SELECT visitorId,
                     visitNumber,
                     visitId,
                     visitStartTime,
                     date,
                     totals,
                     trafficSource,
                     device,
                     geoNetwork,
                     customDimensions,
                     hits,
                     fullVisitorId,
                     userId,
                     channelGrouping,
                     socialEngagementType
              FROM `data-to-insights.ecommerce.web_analytics`
              WHERE date BETWEEN '{date_start_str}' AND '{date_end_str}'
              """
# extract and transform
df = (
    th.extract_data(query_infer, gcp_auth_dict)
    .pipe(th.set_datatypes, dtypes_dict_raw)
)
# load
curr_dtime_str = (
    datetime.now()
    .replace(tzinfo=pytz.utc)
    .astimezone(pytz.timezone('US/Eastern'))
    .strftime('%Y%m%d_%H%M%S')
)
df.pipe(
    load,
    os.path.join(
        raw_data_dir,
        "ga_data",
        (
            f"d2i_ecwa__{date_start_str}__{date_end_str}__"
            f"{curr_dtime_str}.parquet.gzip"
        ),
    )
)
ut.summarize_df(df)
df

Query execution start time = 2023-05-30 16:10:52.018...done at 2023-05-30 16:13:42.154 (170.136 seconds).
Query returned 29,396 rows


,column,dtype,missing
0,visitorId,Int64,29396
1,visitNumber,Int64,0
2,visitId,Int64,0
3,visitStartTime,Int64,0
4,date,string[python],0
5,totals,object,0
6,trafficSource,object,0
7,device,object,0
8,geoNetwork,object,0
9,customDimensions,object,0


,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,<NA>,1,1479672528,1479672528,20161120,"{'visits': 1, 'hits': 5, 'pageviews': 5, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",1167282949320784987,<NA>,Organic Search,Not Socially Engaged
1,<NA>,1,1479289370,1479289370,20161116,"{'visits': 1, 'hits': 8, 'pageviews': 8, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 1, 'minut...",7489798789142578416,<NA>,Direct,Not Socially Engaged
2,<NA>,1,1479429818,1479429818,20161117,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 16, 'minu...",0354024637644335904,<NA>,Organic Search,Not Socially Engaged
3,<NA>,1,1479317439,1479317439,20161116,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'South...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 9, 'minut...",6739790778270031737,<NA>,Organic Search,Not Socially Engaged
4,<NA>,1,1479751482,1479751482,20161121,"{'visits': 1, 'hits': 6, 'pageviews': 3, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 10, 'minu...",9466729361246955500,<NA>,Organic Search,Not Socially Engaged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29391,<NA>,1,1479781678,1479781678,20161121,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Southea...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 18, 'minu...",8253082148281454294,<NA>,Organic Search,Not Socially Engaged
29392,<NA>,1,1479765379,1479765379,20161121,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...",{'referralPath': '/a/google.com/googletopia/di...,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",8271597176638879022,<NA>,Referral,Not Socially Engaged
29393,<NA>,249,1479764387,1479764387,20161121,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",0824839726118485274,<NA>,Organic Search,Not Socially Engaged
29394,<NA>,1,1479541867,1479541867,20161118,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 23, 'minu...",530051687701762318,<NA>,Organic Search,Not Socially Engaged
